In [1]:
from __future__ import print_function
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv("Datasets/mergedChEMBLDF.csv")
data.head()

/home/esben/.virtualenvs/keras2/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,29,31,33,35,37,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,stdInChiKey,1613933_operator,1613933_value,1614063_operator,1614063_value,1614364_operator,1614364_value,1614421_operator,1614421_value,1794557_operator,...,3705123_operator,3705123_value,3705362_operator,3705362_value,3705488_operator,3705488_value,3706373_operator,3706373_value,smiles,smiles_parent
0,RJYFVJPEXKPGNZ-UHFFFAOYSA-N,NaN,NaN,NaN,NaN,NaN,NaN,=,1412.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COc1ccccc1C2CCN(CC2)C(=Nc3ccc(Br)cc3)S,COc1ccccc1C1CCN(C(S)=Nc2ccc(Br)cc2)CC1
1,GNWGGHKGEXNNMK-UHFFFAOYSA-N,NaN,NaN,NaN,NaN,NaN,NaN,=,17782.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cc1ccc(O)c(NC(=O)c2onc(c2)c3ccc(F)cc3)c1,Cc1ccc(O)c(NC(=O)c2cc(-c3ccc(F)cc3)no2)c1
2,LVDHDRPSCLIFHH-RDRPBHBLSA-N,NaN,NaN,NaN,NaN,=,5011.9,=,10000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COc1ccccc1NC(=O)C(=O)N\N=C(/C)\CC(=O)Nc2ccc(OC...,COc1ccccc1NC(=O)C(=O)NN=C(C)CC(=O)Nc1ccc(OCc2c...
3,OJENVBWGZAMKEC-UHFFFAOYSA-N,NaN,NaN,NaN,NaN,NaN,NaN,=,707.9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC(=O)c1ccc(NC(=O)c2oc(cc2)c3ccc(Cl)cc3)cc1,CC(=O)c1ccc(NC(=O)c2ccc(-c3ccc(Cl)cc3)o2)cc1
4,HGUWAIXVEMNODZ-UHFFFAOYSA-N,NaN,NaN,NaN,NaN,NaN,NaN,=,19952.6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cc1nc2ncnn2c(N3CCN(CC3)c4cccc(c4)C(F)(F)F)c1C,Cc1nc2ncnn2c(N2CCN(c3cccc(C(F)(F)F)c3)CC2)c1C


In [3]:
data.columns

Index([u'stdInChiKey', u'1613933_operator', u'1613933_value',
       u'1614063_operator', u'1614063_value', u'1614364_operator',
       u'1614364_value', u'1614421_operator', u'1614421_value',
       u'1794557_operator', u'1794557_value', u'1909116_operator',
       u'1909116_value', u'1909123_operator', u'1909123_value',
       u'1909130_operator', u'1909130_value', u'1909184_operator',
       u'1909184_value', u'1909186_operator', u'1909186_value',
       u'1909187_operator', u'1909187_value', u'1909194_operator',
       u'1909194_value', u'1909195_operator', u'1909195_value',
       u'1909196_operator', u'1909196_value', u'2218781_operator',
       u'2218781_value', u'2328568_operator', u'2328568_value',
       u'3705123_operator', u'3705123_value', u'3705362_operator',
       u'3705362_value', u'3705488_operator', u'3705488_value',
       u'3706373_operator', u'3706373_value', u'smiles', u'smiles_parent'],
      dtype='object')

In [4]:
assays = [1614421,1614364,1613933,1909116,1909123,1909130,1909187,1909186,1909184,1909194,1909195,1909196,2328568,1794557,3706373,3705123,1614063,3705488,3705362,2218781]

train_assays = assays[:10]
test_assays = assays[10:]


In [5]:
a =set(train_assays)
a.intersection(set(test_assays))

set()

In [9]:
data.shape

(65382, 44)

In [7]:
def morgan_fp(smiles):
	mol = Chem.MolFromSmiles(smiles)
	fp = AllChem.GetMorganFingerprintAsBitVect(mol,3, nBits=8192)
	npfp = np.array(list(fp.ToBitString())).astype('int8')
	return npfp

data["morgan"] = data["smiles_parent"].apply(morgan_fp)

In [10]:
#Fingerprint the mols
from rdkit.Chem.EState import Fingerprinter
from rdkit.Chem.AtomPairs import Torsions
#from rdkit.Avalon import pyAvalonTools
from rdkit.Chem import MACCSkeys, AllChem
def combi_fp(smiles):
    mol = Chem.MolFromSmiles(smiles)
    morgan = AllChem.GetMorganFingerprintAsBitVect(mol,4, nBits=8192)
    maccs = MACCSkeys.GenMACCSKeys(mol)
    #avalon = pyAvalonTools.GetAvalonFP(mol, nBits=8192)
    estate = Fingerprinter.FingerprintMol(mol)[0]
    topotorsion = Torsions.GetHashedTopologicalTorsionFingerprint(mol)
    npfp = np.array(list(morgan.ToBitString()) + list(maccs.ToBitString()) + list(estate) + list(topotorsion)  ).astype('int8')
    return npfp


In [11]:
data["combi"] = data["smiles_parent"].apply(combi_fp)

In [13]:
data.columns

Index([u'stdInChiKey', u'1613933_operator', u'1613933_value',
       u'1614063_operator', u'1614063_value', u'1614364_operator',
       u'1614364_value', u'1614421_operator', u'1614421_value',
       u'1794557_operator', u'1794557_value', u'1909116_operator',
       u'1909116_value', u'1909123_operator', u'1909123_value',
       u'1909130_operator', u'1909130_value', u'1909184_operator',
       u'1909184_value', u'1909186_operator', u'1909186_value',
       u'1909187_operator', u'1909187_value', u'1909194_operator',
       u'1909194_value', u'1909195_operator', u'1909195_value',
       u'1909196_operator', u'1909196_value', u'2218781_operator',
       u'2218781_value', u'2328568_operator', u'2328568_value',
       u'3705123_operator', u'3705123_value', u'3705362_operator',
       u'3705362_value', u'3705488_operator', u'3705488_value',
       u'3706373_operator', u'3706373_value', u'smiles', u'smiles_parent',
       u'morgan', u'combi'],
      dtype='object')

In [12]:
data.to_hdf("Datasets/mergedCHEMBLDF.h5","data")

/home/esben/.virtualenvs/keras2/local/lib/python2.7/site-packages/pandas/core/generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['stdInChiKey', '1613933_operator', '1614063_operator', '1614364_operator', '1614421_operator', '1794557_operator', '1909116_operator', '1909116_value', '1909123_operator', '1909123_value', '1909130_operator', '1909130_value', '1909184_operator', '1909184_value', '1909186_operator', '1909186_value', '1909187_operator', '1909187_value', '1909194_operator', '1909194_value', '1909195_operator', '1909195_value', '1909196_operator', '1909196_value', '2218781_operator', '2328568_operator', '3705123_operator', '3705362_operator', '3705488_operator', '3706373_operator', 'smiles', 'smiles_parent', 'morgan', 'combi']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
